In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import GPy

# 创建保存文件夹
output_folder = "uuv_visualizations_sparse_gp_submit_global2d"
os.makedirs(output_folder, exist_ok=True)
seed = 41  # 可以是任意整数
np.random.seed(seed)
# 初始化 GMM 先验分布
means = np.array([[5, 10], [15, 5], [10, 15], [5, 5], [15, 15]])
covariances = [
    [[3, 0.5], [0.5, 3]],
    [[4, -0.5], [-0.5, 3.5]],
    [[2.5, 0.8], [0.8, 2.5]],
    [[3.5, 1], [1, 3]],
    [[3, 0], [0, 3]]
]
weights = [0.2, 0.3, 0.1, 0.25, 0.15]

# 创建网格和计算概率密度
x_grid, y_grid = np.meshgrid(np.linspace(0, 20, 50), np.linspace(0, 20, 50))
grid_points = np.column_stack([x_grid.ravel(), y_grid.ravel()])

gmm_probs = np.zeros(grid_points.shape[0])
for mean, cov, weight in zip(means, covariances, weights):
    rv = multivariate_normal(mean, cov)
    gmm_probs += weight * rv.pdf(grid_points)

# 初始化稀疏 GP
kernel = GPy.kern.RBF(input_dim=2, variance=1.0, lengthscale=1.0)
num_inducing = 30
inducing_points = grid_points[np.random.choice(grid_points.shape[0], num_inducing, replace=False)]
sparse_gp_model = GPy.models.SparseGPRegression(grid_points, gmm_probs[:, None], kernel, Z=inducing_points)
sparse_gp_model.optimize()
# 允许优化诱导点的位置
sparse_gp_model.Z.optimize = True
sparse_gp_model.optimize(messages=False, max_iters=1000)


# 初始化 UUV 和其他变量
num_uuv = 3
# uuv_positions = np.zeros((num_uuv, 2))+ np.random.uniform(-0.6, 0.6, (num_uuv, 2))
uuv_positions = np.zeros((num_uuv, 2))+ np.random.uniform(-2, 2, (num_uuv, 2))
uuv_initial_positions = np.copy(uuv_positions)  # 复制初始状态，防止后续更新影响
uuv_status = [False] * num_uuv
uuv_targets = [None] * num_uuv
observed_points = []
observed_values = []
assigned_targets = set()  # 已分配目标点集合

# 参数设置
max_steps = 50
max_distance = 1.0  # UUV 每步最大移动距离
kappa = 1.0  # UCB 中的探索权重
target_radius = 3.0
target_position = np.array([14.0, 16.0])  # 目标实际位置
num_interest_points = 5  # 每轮兴趣点数量
min_distance_between_points = 3.0  # 兴趣点之间的最小距离
# uuv_energy = np.full(num_uuv, 1000.0)  # 假设每个 UUV 初始能源为 100
uuv_energy = np.array([1000.0, 800.0, 1200.0])  # 设定每个 UUV 初始能源不同  # 假设每个 UUV 初始能源为 100


# 定义函数
def sense_target(current_point, target_position, target_radius):
    distance = np.linalg.norm(current_point - target_position)
    return 1 if distance <= target_radius else -0.11

def find_global_interest_points(grid_points, sparse_gp_model, kappa, num_points, min_distance):
    mu, var = sparse_gp_model.predict(grid_points)
    sigma = np.sqrt(var)
    ucb = mu.flatten() + kappa * sigma.flatten()

    # 排序并选择兴趣点
    sorted_indices = np.argsort(ucb)[::-1]  # 按 UCB 从大到小排序
    selected_points = []
    selected_ucb = []
    selected_sigma = []

    for idx in sorted_indices:
        candidate_point = grid_points[idx]
        # 检查与已选点的最小距离
        if all(np.linalg.norm(candidate_point - np.array(p)) >= min_distance for p in selected_points):
            selected_points.append(candidate_point)
            selected_ucb.append(ucb[idx])
            selected_sigma.append(sigma[idx])
        if len(selected_points) == num_points:
            break

    return np.array(selected_points), np.array(selected_ucb), np.array(selected_sigma)

def auction_points(interest_points, interest_ucb, interest_sigma, uuv_positions, uuv_status, uuv_energy, assigned_targets, min_distance=3.0):
    bids = {}
    assigned_uuvs = set()
    assigned_points = set()

    for idx, point in enumerate(interest_points):
        # 检查当前兴趣点是否在已分配目标的附近
        if any(np.linalg.norm(point - np.array(assigned_point)) < min_distance for assigned_point in assigned_targets):
            continue  # 跳过已分配目标附近的点

        for uuv_id, position in enumerate(uuv_positions):
            if not uuv_status[uuv_id] and uuv_energy[uuv_id] > 0:  # 只考虑未分配任务且有足够能源的UUV
                distance = np.linalg.norm(position - point)
                information_gain = interest_sigma[idx]
                ucb_gain = interest_ucb[idx]
                
                # 考虑能源消耗，设定一个能耗系数 0.5 * distance
                energy_cost = 0.5 * distance
                
                # 调整成本计算方式，平衡距离、信息增益、UCB收益和能源消耗
                cost = 10 * distance - 2.0 * information_gain + 1.0 * ucb_gain + 5.0 * energy_cost
                
                if cost < uuv_energy[uuv_id]:  # 确保 UUV 有足够能源完成任务
                    if idx not in bids:
                        bids[idx] = []
                    bids[idx].append((uuv_id, cost))
    
    assignments = {}
    for point_idx, bid_list in bids.items():
        bid_list.sort(key=lambda x: x[1])  # 按成本排序
        
        for uuv_id, cost in bid_list:
            if point_idx not in assigned_points and uuv_id not in assigned_uuvs:
                assignments[point_idx] = uuv_id
                assigned_points.add(point_idx)
                assigned_uuvs.add(uuv_id)
                uuv_status[uuv_id] = True  # 标记UUV为已分配任务
                assigned_targets.append(interest_points[point_idx])  # 记录已分配的兴趣点
                
                # 更新 UUV 能源
                uuv_energy[uuv_id] -= cost
                
                break  # 确保每个兴趣点只分配给一个UUV
    
    return assignments



# 在每轮更新GP模型后，优化诱导点
def update_gp_with_observations(grid_points, gmm_probs, observed_points, observed_values, gp_model):
    if len(observed_points) > 0:
        observed_points_array = np.array(observed_points)
        updated_points = np.vstack([grid_points, observed_points_array])
        updated_values = np.hstack([gmm_probs, observed_values])
    else:
        updated_points = grid_points
        updated_values = gmm_probs

    gp_model.set_XY(updated_points, updated_values[:, None])
    
    gp_model.Z.optimize = True
    gp_model.optimize(messages=False, max_iters=1000)  

    return gp_model

import matplotlib.pyplot as plt
from matplotlib import rcParams

# 设置 IEEE 风格
rcParams['font.family'] = 'Times New Roman'
rcParams['font.size'] = 14  # 增大字体
rcParams['axes.labelsize'] = 16  # 增大坐标轴标签字体
rcParams['axes.titlesize'] = 18  # 增大标题字体
rcParams['xtick.labelsize'] = 14  # 增大刻度字体
rcParams['ytick.labelsize'] = 14  # 增大刻度字体
rcParams['legend.fontsize'] = 14  # 增大图例字体
rcParams['figure.titlesize'] = 20  # 增大图形标题字体

# 定义 UUV 颜色和轨迹颜色
uuv_colors = ['red', 'green', 'blue']  # 每个 UUV 的颜色
trail_colors = ['darkred', 'darkgreen', 'darkblue']  # 每个 UUV 的轨迹颜色
# 记录 UUV 轨迹
positions = {f"auv{i}": [] for i in range(num_uuv)}

def save_visualization(step_count, uuv_positions, observed_points, interest_points, target_position, gp_model, grid_points, x_grid, assignments):
    # 创建图形
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # 预测 GP 均值和方差
    mu, var = gp_model.predict(grid_points)
    gp_mu = mu.reshape(x_grid.shape)
    gp_var = var.reshape(x_grid.shape)

    # 绘制 GP 均值的热图
    im = ax.imshow(gp_mu, extent=(0, 20, 0, 20), origin='lower', cmap='coolwarm', alpha=0.8)
    cbar = plt.colorbar(im, ax=ax, label='SGP Mean')
    cbar.ax.tick_params(labelsize=14)  # 设置 colorbar 字体大小

    # 绘制观测点（灰色）
    if observed_points:
        obs_points = np.array(observed_points)
        ax.scatter(obs_points[:, 0], obs_points[:, 1], c='gray', s=50, label='Observed Points', edgecolor='black', linewidth=0.5)

    # 绘制兴趣点
    ax.scatter(interest_points[:, 0], interest_points[:, 1], c='purple', s=100, label='Interest Points', marker='x', linewidth=2)

    # 绘制目标点
    ax.scatter(target_position[0], target_position[1], marker='*', c='yellow', s=200, label='Target', edgecolor='black', linewidth=0.5)

    # 绘制 UUV 位置和轨迹
    for i in range(num_uuv):
        # 绘制 UUV 当前位置
        ax.scatter(uuv_positions[i, 0], uuv_positions[i, 1], c=uuv_colors[i], s=100, label=f'UUV {i}', edgecolor='black', linewidth=0.5)
        
        # 绘制 UUV 轨迹
        if len(positions[f"auv{i}"]) > 1:
            trail = np.array(positions[f"auv{i}"])
            ax.plot(trail[:, 0], trail[:, 1], c=trail_colors[i], linestyle='-', linewidth=1.5, alpha=0.7, label=f'UUV {i} Trail')

    # 绘制 UUV 到兴趣点的路径
    for point_idx, uuv_id in assignments.items():
        ax.plot([uuv_positions[uuv_id, 0], interest_points[point_idx, 0]],
                [uuv_positions[uuv_id, 1], interest_points[point_idx, 1]],
                c=uuv_colors[uuv_id], linestyle='--', linewidth=1.5, alpha=0.7,
                label=f'UUV {uuv_id} → Interest {point_idx}')

    # 设置标题和标签
    # ax.set_title(f"Step {step_count}", fontsize=18, pad=20)
    ax.set_xlabel("X Coordinate", fontsize=16)
    ax.set_ylabel("Y Coordinate", fontsize=16)

    # 添加图例（左上角）
    ax.legend(loc='upper left', fontsize=14, framealpha=0.9)

    # 调整布局
    plt.tight_layout()

    # 保存图形
    plt.savefig(os.path.join(output_folder, f"step_{step_count:03d}.png"), dpi=300, bbox_inches='tight')
    plt.close()
# 主循环
found_target = False
assigned_targets = []  # 初始化已分配目标点集合
# 记录UUV目标点的列表
uuv_targets_per_step = []  # 用于存储每轮 UUV 的目标点

for step in range(max_steps):
    if found_target:
        print("Target found, stopping exploration.")
        break

    print(f"\nRound {step + 1}")

    # 重新计算兴趣点
    interest_points, interest_ucb, interest_sigma = find_global_interest_points(
        grid_points, sparse_gp_model, kappa, num_points=num_interest_points, min_distance=min_distance_between_points
    )
    assignments = auction_points(
        interest_points, interest_ucb, interest_sigma, uuv_positions, uuv_status, uuv_energy, assigned_targets, min_distance=min_distance_between_points
    )

    # **添加此部分: 记录每轮UUV目标**
    step_targets = {}  # 记录当前轮次UUV目标点
    for point_idx, uuv_id in assignments.items():
        step_targets[f"UUV_{uuv_id}"] = interest_points[point_idx].tolist()
        uuv_targets_per_step.append(step_targets)  # 存入列表

# for step in range(max_steps):
#     if found_target:
#         print("Target found, stopping exploration.")
#         break

#     print(f"\nRound {step + 1}")

#     # 更新兴趣点
#     interest_points, interest_ucb, interest_sigma = find_global_interest_points(grid_points, sparse_gp_model, kappa, num_points=num_interest_points, min_distance=min_distance_between_points)
#     assignments = auction_points(interest_points, interest_ucb, interest_sigma, uuv_positions, uuv_status, uuv_energy, assigned_targets, min_distance=min_distance_between_points)

#     for point_idx, uuv_id in assignments.items():
        uuv_targets[uuv_id] = interest_points[point_idx]
        uc_value = float(interest_ucb[point_idx])  # 确保是标量
        ig_value = float(interest_sigma[point_idx])  # 确保是标量
        print(f"UUV {uuv_id} assigned to interest point {interest_points[point_idx]} with UCB {uc_value:.2f} and Information Gain {ig_value:.2f}")

    for i in range(num_uuv):
        if uuv_targets[i] is not None:
            target = uuv_targets[i]
            distance = np.linalg.norm(uuv_positions[i] - target)

            if distance <= max_distance:
                uuv_positions[i] = target
                observed_value = sense_target(target, target_position, target_radius)
                observed_points.append(target)
                observed_values.append(observed_value)

                if observed_value == 1:
                    found_target = True
                    print(f"UUV {i} found the target at {target}.")

                uuv_targets[i] = None
                uuv_status[i] = False  # 重置UUV状态为未分配任务
                print(f"UUV {i} reached point: {target} with observation value: {observed_value}")
            else:
                direction = (target - uuv_positions[i]) / distance
                uuv_positions[i] += direction * max_distance

                observed_value = sense_target(uuv_positions[i], target_position, target_radius)
                observed_points.append(uuv_positions[i].copy())
                observed_values.append(observed_value)

                if observed_value == 1:
                    found_target = True
                    print(f"UUV {i} found the target at {uuv_positions[i]}.")

    # 更新 GP 模型
    sparse_gp_model = update_gp_with_observations(grid_points, gmm_probs, observed_points, observed_values, sparse_gp_model)
    save_visualization(step, uuv_positions, observed_points, interest_points, target_position, sparse_gp_model, grid_points, x_grid, assignments)

print("Simulation completed.")



Round 1
UUV 1 assigned to interest point [15.10204082  4.89795918] with UCB 0.01 and Information Gain 0.00
UUV 0 assigned to interest point [4.89795918 4.89795918] with UCB 0.01 and Information Gain 0.00
UUV 2 assigned to interest point [4.89795918 9.79591837] with UCB 0.01 and Information Gain 0.00

Round 2

Round 3

Round 4

Round 5

Round 6

Round 7

Round 8

Round 9

Round 10
UUV 0 reached point: [4.89795918 4.89795918] with observation value: -0.11

Round 11
UUV 0 assigned to interest point [13.87755102 15.51020408] with UCB 0.02 and Information Gain 0.01

Round 12

Round 13

Round 14

Round 15
UUV 2 reached point: [4.89795918 9.79591837] with observation value: -0.11

Round 16

Round 17
UUV 2 assigned to interest point [17.14285714  2.04081633] with UCB 0.02 and Information Gain 0.01

Round 18
UUV 1 reached point: [15.10204082  4.89795918] with observation value: -0.11

Round 19
UUV 1 assigned to interest point [ 9.79591837 15.51020408] with UCB 0.02 and Information Gain 0.02

R

In [36]:
import json
import os

def save_simulation_results_2d(uuv_initial_positions, uuv_targets_per_step, means, target_position, output_folder):
    """
    保存 2D UUV 初始位置、每步目标分配、GMM 均值和目标位置到 JSON 文件。
    """
    results = {
        "uuv_initial_positions": uuv_initial_positions.tolist(),
        "uuv_targets_per_step": uuv_targets_per_step,
        "gmm_means": means.tolist(),
        "target_position": target_position.tolist()
    }
    
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)
    
    # 保存 JSON 文件
    with open(os.path.join(output_folder, "simulation_results_2d.json"), "w") as f:
        json.dump(results, f, indent=4)
    
    print("2D Simulation results saved to simulation_results_2d.json")

# 调用保存函数
save_simulation_results_2d(uuv_initial_positions, uuv_targets_per_step, means, target_position, output_folder)


2D Simulation results saved to simulation_results_2d.json


In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import GPy
from mpl_toolkits.mplot3d import Axes3D

# --- 1. 保存文件夹 ---
output_folder = "uuv_visualizations_sparse_gp_submit_global_3d2"
os.makedirs(output_folder, exist_ok=True)
seed = 41  # 可以是任意整数
np.random.seed(seed)
# --- 2. 初始化 GMM 先验分布 ---
means = np.array([
    [5, 5, 5],    # 左下角
    [5, 15, 15],  # 左上角
    [15, 5, 15],  # 右下角
    [15, 15, 5],  # 右上角
    [10, 10, 10]  # 中心点
])

covariances = [
    [[4, 0.3, 0.2], [0.3, 4, 0.2], [0.2, 0.2, 4]],  # 左下角
    [[3.5, 0.2, 0.1], [0.2, 3.5, 0.2], [0.1, 0.2, 3.5]],  # 左上角
    [[3, -0.3, 0.1], [-0.3, 3, 0.2], [0.1, 0.2, 3]],  # 右下角
    [[3.5, 0.4, 0.3], [0.4, 3.5, 0.2], [0.3, 0.2, 3.5]],  # 右上角
    [[4, 0, 0.1], [0, 4, 0.1], [0.1, 0.1, 4]]  # 中心点
]

weights = [0.2, 0.2, 0.2, 0.2, 0.2]  # 均等权重分布


# 创建网格
x_grid, y_grid, z_grid = np.meshgrid(np.linspace(0, 20, 20), np.linspace(0, 20, 20), np.linspace(0, 20, 20))
grid_points = np.column_stack([x_grid.ravel(), y_grid.ravel(), z_grid.ravel()])

# 计算 GMM 概率密度
gmm_probs = np.zeros(grid_points.shape[0])
for mean, cov, weight in zip(means, covariances, weights):
    rv = multivariate_normal(mean, cov)
    gmm_probs += weight * rv.pdf(grid_points)

# --- 3. 初始化稀疏 GP ---
# kernel = GPy.kern.RBF(input_dim=3, variance=1.0, lengthscale=1.0)
kernel = GPy.kern.Matern32(input_dim=3, variance=1.0, lengthscale=1.0)

num_inducing = 300
inducing_points = grid_points[np.random.choice(grid_points.shape[0], num_inducing, replace=False)]
# sparse_gp_model = GPy.models.SparseGPRegression(grid_points, gmm_probs[:, None], kernel, Z=inducing_points)
# sparse_gp_model.Z.optimize = True  
# sparse_gp_model.optimize(messages=False, max_iters=500)
sparse_gp_model = GPy.models.SparseGPRegression(grid_points, gmm_probs[:, None], kernel, Z=inducing_points)
sparse_gp_model.inference_method = GPy.inference.latent_function_inference.FITC()
sparse_gp_model.Z.optimize = True  
sparse_gp_model.optimize(messages=True, max_iters=20)

# --- 4. 初始化 UUV 和变量 ---
num_uuv = 3
uuv_positions = np.zeros((num_uuv, 3)) + np.random.uniform(-2, 2, (num_uuv, 3))  # 随机初始化 UUV 位置
uuv_initial_positions = np.copy(uuv_positions)  # 保存初始位置
uuv_status = [False] * num_uuv
uuv_targets = [None] * num_uuv
observed_points = []
observed_values = []
assigned_targets = set()

# --- 5. 参数设置 ---
max_steps = 50
max_distance = 2.0
kappa = 0.001
target_radius = 3.0
target_position = np.array([10, 10, 10])
num_interest_points = 10
min_distance_between_points = 5.0

# --- 6. 函数定义 ---
def sense_target(current_point, target_position, target_radius):
    return 1 if np.linalg.norm(current_point - target_position) <= target_radius else -0.1

def find_global_interest_points(grid_points, sparse_gp_model, kappa, num_points, min_distance):
    mu, var = sparse_gp_model.predict(grid_points)
    sigma = np.sqrt(var)
    ucb = mu.flatten() + kappa * sigma.flatten()

    sorted_indices = np.argsort(ucb)[::-1]
    selected_points, selected_ucb, selected_sigma = [], [], []

    for idx in sorted_indices:
        candidate_point = grid_points[idx]
        if all(np.linalg.norm(candidate_point - np.array(p)) >= min_distance for p in selected_points):
            selected_points.append(candidate_point)
            selected_ucb.append(ucb[idx])
            selected_sigma.append(sigma[idx])
        if len(selected_points) == num_points:
            break

    return np.array(selected_points), np.array(selected_ucb), np.array(selected_sigma)

def auction_points(interest_points, interest_ucb, interest_sigma, uuv_positions, uuv_status, assigned_targets, min_distance=3.0):
    bids = {}
    assigned_uuvs = set()
    assigned_points = set()

    for idx, point in enumerate(interest_points):
        if any(np.linalg.norm(point - np.array(assigned_point)) < min_distance for assigned_point in assigned_targets):
            continue
        for uuv_id, position in enumerate(uuv_positions):
            if not uuv_status[uuv_id]:
                distance = np.linalg.norm(position - point)
                information_gain = interest_sigma[idx]
                ucb_gain = interest_ucb[idx]
                cost = 10 * distance - 2.0 * information_gain + 1.0 * ucb_gain
                if idx not in bids:
                    bids[idx] = []
                bids[idx].append((uuv_id, cost))

    assignments = {}
    for point_idx, bid_list in bids.items():
        bid_list.sort(key=lambda x: x[1])
        for uuv_id, cost in bid_list:
            if point_idx not in assigned_points and uuv_id not in assigned_uuvs:
                assignments[point_idx] = uuv_id
                assigned_points.add(point_idx)
                assigned_uuvs.add(uuv_id)
                uuv_status[uuv_id] = True
                assigned_targets.add(tuple(interest_points[point_idx])) 
                break

    return assignments

def update_gp_with_observations(grid_points, gmm_probs, observed_points, observed_values, gp_model):
    if len(observed_points) > 0:
        observed_points_array = np.array(observed_points)
        updated_points = np.vstack([grid_points, observed_points_array])
        updated_values = np.hstack([gmm_probs, observed_values])
    else:
        updated_points = grid_points
        updated_values = gmm_probs

    gp_model.set_XY(updated_points, updated_values[:, None])
    gp_model.Z.optimize = True  
    gp_model.optimize(messages=False, max_iters=10)  

    return gp_model

def save_visualization(step_count, uuv_positions, observed_points, interest_points, target_position, gp_model, grid_points, x_grid, assignments):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    mu, _ = gp_model.predict(grid_points)
    ax.scatter(grid_points[:, 0], grid_points[:, 1], grid_points[:, 2], c=mu, cmap='coolwarm', alpha=0.6, s=5)

    if observed_points:
        obs_points = np.array(observed_points)
        ax.scatter(obs_points[:, 0], obs_points[:, 1], obs_points[:, 2], c='green', s=50, label='Observed Points')
    ax.scatter(interest_points[:, 0], interest_points[:, 1], interest_points[:, 2], c='purple', s=100, label='Interest Points', marker='x')
    ax.scatter(target_position[0], target_position[1], target_position[2], marker='*', c='yellow', s=200, label='Target')
    ax.scatter(uuv_positions[:, 0], uuv_positions[:, 1], uuv_positions[:, 2], c='blue', s=100, label='UUVs')

    colors = ['r', 'g', 'b', 'c', 'm', 'y']
    for point_idx, uuv_id in assignments.items():
        ax.plot([uuv_positions[uuv_id, 0], interest_points[point_idx, 0]],
                [uuv_positions[uuv_id, 1], interest_points[point_idx, 1]],
                [uuv_positions[uuv_id, 2], interest_points[point_idx, 2]],
                c=colors[uuv_id % len(colors)], linestyle='--', linewidth=1.5,
                label=f'UUV {uuv_id} → Interest {point_idx}')

    # ax.set_title(f"Step {step_count}")
    ax.legend(loc='lower left')
    plt.savefig(os.path.join(output_folder, f"step_{step_count:03d}.png"))
    plt.close()

# --- 7. 主循环 ---
found_target = False
assigned_targets = set()

# 在主循环开始前，初始化变量以记录UUV拍卖目标
uuv_targets_per_step = []  # 用于保存每轮的UUV目标位置

for step in range(max_steps):
    if found_target:
        print("Target found, stopping exploration.")
        break

    print(f"\nRound {step + 1}")

    # 重新计算兴趣点
    interest_points, interest_ucb, interest_sigma = find_global_interest_points(
        grid_points, sparse_gp_model, kappa, num_points=num_interest_points, min_distance=min_distance_between_points)
    assignments = auction_points(interest_points, interest_ucb, interest_sigma, 
                                 uuv_positions, uuv_status, assigned_targets, min_distance=min_distance_between_points)

    # 保存当前轮次每个UUV的目标位置
    step_targets = {}
    for point_idx, uuv_id in assignments.items():
        step_targets[f"UUV_{uuv_id}"] = interest_points[point_idx].tolist()
        uuv_targets_per_step.append(step_targets)

    # 原有UUV移动与观测逻辑...
        uuv_targets[uuv_id] = interest_points[point_idx]
        uc_value = float(interest_ucb[point_idx])
        ig_value = float(interest_sigma[point_idx])
        print(f"UUV {uuv_id} assigned to interest point {interest_points[point_idx]} with UCB {uc_value:.9f} and Information Gain {ig_value:.2f}")

    for i in range(num_uuv):
        if uuv_targets[i] is not None:
            target = uuv_targets[i]
            distance = np.linalg.norm(uuv_positions[i] - target)

            if distance <= max_distance:
                uuv_positions[i] = target
                observed_value = sense_target(target, target_position, target_radius)
                observed_points.append(target)
                observed_values.append(observed_value)

                if observed_value == 1:
                    found_target = True
                    print(f"UUV {i} found the target at {target}.")

                uuv_targets[i] = None
                uuv_status[i] = False
                print(f"UUV {i} reached point: {target} with observation value: {observed_value}")
            else:
                direction = (target - uuv_positions[i]) / distance
                uuv_positions[i] += direction * max_distance

                observed_value = sense_target(uuv_positions[i], target_position, target_radius)
                observed_points.append(uuv_positions[i].copy())
                observed_values.append(observed_value)

                if observed_value == 1:
                    found_target = True
                    print(f"UUV {i} found the target at {uuv_positions[i]}.")

    # 每轮迭代后更新 GP 并重新优化诱导点
    if step % 5 == 0:
        sparse_gp_model = update_gp_with_observations(grid_points, gmm_probs, observed_points, observed_values, sparse_gp_model)
    save_visualization(step, uuv_positions, observed_points, interest_points, target_position, sparse_gp_model, grid_points, x_grid, assignments)

print("Simulation completed.")



Round 1
UUV 0 assigned to interest point [14.73684211  4.21052632 14.73684211] with UCB 0.001236127 and Information Gain 0.00
UUV 1 assigned to interest point [14.73684211 14.73684211  5.26315789] with UCB 0.001116768 and Information Gain 0.00
UUV 2 assigned to interest point [ 5.26315789 14.73684211 14.73684211] with UCB 0.001094000 and Information Gain 0.00

Round 2

Round 3

Round 4

Round 5

Round 6

Round 7

Round 8

Round 9

Round 10

Round 11
UUV 0 reached point: [14.73684211  4.21052632 14.73684211] with observation value: -0.1
UUV 1 reached point: [14.73684211 14.73684211  5.26315789] with observation value: -0.1

Round 12
UUV 1 assigned to interest point [5.26315789 5.26315789 5.26315789] with UCB 0.001898981 and Information Gain 0.00
UUV 0 assigned to interest point [9.47368421 9.47368421 9.47368421] with UCB 0.001749925 and Information Gain 0.00
UUV 2 reached point: [ 5.26315789 14.73684211 14.73684211] with observation value: -0.1

Round 13
UUV 2 assigned to interest poin

In [ ]:
import json
import os

def save_simulation_results(uuv_initial_positions, uuv_targets_per_step, means, target_position, output_folder):
    """
    保存UUV初始位置、拍卖后的目标位置、GMM均值和目标位置到JSON文件。
    """
    results = {
        "uuv_initial_positions": uuv_initial_positions.tolist(),
        "uuv_targets_per_step": uuv_targets_per_step,
        "gmm_means": means.tolist(),
        "target_position": target_position.tolist()
    }
    
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)
    
    # 保存JSON文件
    with open(os.path.join(output_folder, "simulation_results.json"), "w") as f:
        json.dump(results, f, indent=4)
    print("Simulation results saved to simulation_results.json")

# 调用保存函数
save_simulation_results(uuv_initial_positions, uuv_targets_per_step, means, target_position, output_folder)

Simulation results saved to simulation_results.json


: 

In [2]:
import numpy as np

In [39]:
yaw_angles = np.unwrap(np.array([-3.4906585]), period=np.pi)


In [ ]:
yaw_angles

array([-3.4906585])

: 

In [32]:
import numpy as np

# 确保角度数据在 -180° 到 180° 之间的连续变化
angular_velocity_swapped = np.degrees(np.unwrap(np.radians([-200]),period=np.pi))


In [33]:
angular_velocity_swapped

array([-200.])

In [34]:
np.radians([-200])

array([-3.4906585])